In [1]:
import torch
import torch.nn as nn
import visdom
import torch.nn.init
from torch.utils.data import Dataset, DataLoader

import numpy as np

device = 'cpu'

In [2]:
vis = visdom.Visdom()
vis2 = visdom.Visdom()
vis3 = visdom.Visdom()
vis.close(env='main')

Setting up a new session...
Setting up a new session...
Setting up a new session...


''

In [3]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))
accuracy_plt = vis2.line(Y=torch.Tensor(1).zero_(), opts=dict(title='accuracy', legend=['accuracy'], showlegend=True))
test_accuracy_plt = vis3.line(Y=torch.Tensor(1).zero_(), opts=dict(title='test_accuracy', legend=['test_acc'], showlegend=True))


In [4]:
def loss_tracker(plot, loss_value, num):
    vis.line(X=num,
             Y=loss_value,
             win=plot,
             update='append'
             )
def accuracy_tracker(plot, accurcay_value, num):
    vis2.line(X=num,
             Y=accurcay_value,
             win=plot,
             update='append'
             )
def test_accuracy_tracker(plot, accurcay_value, num):
    vis3.line(X=num,
             Y=accurcay_value,
             win=plot,
             update='append'
             )

In [5]:
learning_rate = 0.001
training_epochs = 20
batch_size = 64

In [6]:
np_dataset = np.load('./data/np_dataset_split.npz')
data = torch.from_numpy(np_dataset['data']).float()
targets = torch.from_numpy(np_dataset['targets']).long()
test_data = torch.from_numpy(np_dataset['test_data']).float()
test_targets = torch.from_numpy(np_dataset['test_targets']).long()

In [7]:
class Custom_Dataset(Dataset):

    def __init__(self, data, targets):
        self.data = data
        self.targets = targets

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        X = self.data[index]
        Y = self.targets[index]
        return X, Y

In [8]:
train_dataset = Custom_Dataset(data=data, targets=targets)
test_dataset = Custom_Dataset(data=test_data, targets=test_targets)

In [9]:
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          drop_last=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=True,
                         drop_last=True)

In [10]:
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=16, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=5, stride=2)
        )

        self.fc1 = nn.Linear(in_features=32 * 13 * 21, out_features=11, bias=True)
        self.relu = nn.ReLU()
        

        torch.nn.init.xavier_uniform_(self.fc1.weight)
        


    def forward(self, inputs):
        output = self.layer1(inputs)
        output = self.layer2(output)


        output = output.view(output.size(0), -1)
        output = self.fc1(output)
        return output


In [11]:
model = CNN().to(device)

In [12]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [13]:
total_batch = len(train_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    correct = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost /total_batch
        
    with torch.no_grad():
        X_val = train_dataset.data.view(len(data), 4, 257, 382).float().to(device)
        Y_val = train_dataset.targets.to(device)

        prediction = model(X_val)

        correct_prediction = torch.argmax(prediction, 1) == Y_val
        accuracy = correct_prediction.float().mean()
        
        X_test = test_dataset.data.view(len(test_data), 4, 257, 382).float().to(device)
        Y_test = test_dataset.targets.to(device)

        test_prediction = model(X_test)

        test_correct_prediction = torch.argmax(test_prediction, 1) == Y_test
        test_accuracy = test_correct_prediction.float().mean()
        
        
    print('[Epoch : {:>4}] cost = {:>.9} accuracy = {:>3} test_accuracy = {:>3}'.format(epoch + 1, avg_cost, accuracy.item(), test_accuracy.item()))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    accuracy_tracker(accuracy_plt, torch.Tensor([accuracy]), torch.Tensor([epoch]))
    test_accuracy_tracker(test_accuracy_plt, torch.Tensor([test_accuracy]), torch.Tensor([epoch]))
print('Learning finished')

[Epoch :    1] cost = 2.82953262 accuracy = 0.13249999284744263 test_accuracy = 0.08500000089406967
[Epoch :    2] cost = 2.38229895 accuracy = 0.1287499964237213 test_accuracy = 0.0949999988079071
[Epoch :    3] cost = 2.35839701 accuracy = 0.15000000596046448 test_accuracy = 0.07500000298023224
[Epoch :    4] cost = 2.31975436 accuracy = 0.16124999523162842 test_accuracy = 0.0949999988079071
[Epoch :    5] cost = 2.27084637 accuracy = 0.2150000035762787 test_accuracy = 0.12999999523162842
[Epoch :    6] cost = 2.2084465 accuracy = 0.2787500023841858 test_accuracy = 0.18000000715255737
[Epoch :    7] cost = 2.12201715 accuracy = 0.29374998807907104 test_accuracy = 0.13500000536441803
[Epoch :    8] cost = 2.05151868 accuracy = 0.30250000953674316 test_accuracy = 0.2199999988079071
[Epoch :    9] cost = 1.96309757 accuracy = 0.32875001430511475 test_accuracy = 0.18000000715255737
[Epoch :   10] cost = 1.89732838 accuracy = 0.38499999046325684 test_accuracy = 0.19499999284744263
[Epoch 

In [14]:
with torch.no_grad():
    X_test = test_dataset.data.view(len(test_data), 4, 257, 382).float().to(device)
    Y_test = test_dataset.targets.to(device)

    prediction = model(X_test)

    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

Accuracy :  0.16500000655651093
